In [ ]:
!pip install pytube


In [ ]:
! pip install git+https://github.com/openai/whisper.git


In [ ]:
! pip install jiwer

JiWER is a simple and fast python package to evaluate an automatic speech recognition system.

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

#OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

True

In [2]:
from pytube import YouTube

In [ ]:
link='https://www.youtube.com/watch?v=8aeSjwSUcAc'

In [4]:
try: 
    # object creation using YouTube
    yt = YouTube(link) 
except: 
    print("Connection Error")

In [5]:
yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: 

In [7]:
stream = yt.streams.get_by_itag(139)

In [8]:
stream.download('',"lecuninterview.mp4")

'/workspaces/RAG-langchain-ollama/lecuninterview.mp4'

In [10]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

/home/codespace/.python/current/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

we got the result in a form of a raw text

In [ ]:
with open ('text.txt', 'w') as file:  
    file.write(result['text']) 

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain


In [ ]:
from langchain import OpenAI
llm = OpenAI(temperature=0)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )

In [ ]:
with open('text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
docs

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [ ]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

In [ ]:
chain.llm_chain.prompt.template

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [ ]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS: AND TRANSLATE IT TO FRENCH"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [ ]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=2000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)